In [61]:
from surprise import Dataset
from surprise import Reader
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import time

In [3]:
df_r = pd.read_csv("databases/ratings.csv")
df_b = pd.read_csv("databases/books.csv")
display(df_r.head())
display(df_b.head())

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_r[['user_id', 'book_id', 'rating']], reader)
print(data)

In [19]:
#dir(data)
data.df

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [6]:
from surprise import SVD
from surprise.model_selection import cross_validate

In [7]:
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8535  0.8522  0.8523  0.8527  0.0006  
MAE (testset)     0.6663  0.6656  0.6654  0.6658  0.0004  
Fit time          108.22  108.24  108.98  108.48  0.35    
Test time         23.07   22.70   20.07   21.95   1.34    


{'test_rmse': array([0.85346169, 0.8522269 , 0.8522646 ]),
 'test_mae': array([0.6662949 , 0.66562864, 0.66536016]),
 'fit_time': (108.21944952011108, 108.24445509910583, 108.98258352279663),
 'test_time': (23.07054114341736, 22.703975677490234, 20.07151436805725)}

In [8]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [9]:
svd.predict(uid=10, iid=100)

Prediction(uid=10, iid=100, r_ui=None, est=3.7233741923039445, details={'was_impossible': False})

In [88]:
import difflib
import random

def get_book_id(book_title, metadata):
    
    """
    Gets the book ID for a book title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(metadata['title'].values)
    closest_titles = difflib.get_close_matches(book_title, existing_titles)
    book_id = metadata[metadata['title'] == closest_titles[0]]['book_id'].values[0]
    return book_id

def get_book_info(book_id, metadata):
    
    """
    Returns some basic information about a book given the book id and the metadata dataframe.
    """
    
    book_info = metadata[metadata['book_id'] == book_id][['book_id', 'isbn', 
                                                    'authors', 'title', 'original_title']]
    return book_info.to_dict(orient='records')

def predict_review(user_id, book_title, model, metadata):
    
    """
    Predicts the review (on a scale of 1-5) that a user would assign to a specific book. 
    """
    
    book_id = get_book_id(book_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=book_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=4, verbose=0, nb_wanted_books=1):
    
    """
    Generates a book recommendation for a user based on a rating threshold. Only
    books with a predicted rating at or above the threshold will be recommended
    """
    
    book_titles = list(metadata['title'].values)
    random.shuffle(book_titles)
    
    # test
    book_list = {"book_info": [], "rating": []}
    
    for book_title in tqdm(book_titles):
        rating = predict_review(user_id, book_title, model, metadata)
        if rating >= thresh:
            book_id = get_book_id(book_title, metadata)
            # test
            #return get_book_info(book_id, metadata)
            book_list["book_info"].append(get_book_info(book_id, metadata))
            book_list["rating"].append(rating)
        if len(book_list["rating"]) >= nb_wanted_books:
            break
    return book_list

In [89]:
%%time
ret = generate_recommendation(1000, svd, df_b, verbose=1)
print(type(ret))
print(len(ret))
print(ret)


<class 'dict'>
2
{'book_info': [[{'book_id': 9076, 'isbn': '402366174', 'authors': 'The Church of Jesus Christ of Latter-day Saints', 'title': 'Preach My Gospel: A Guide To Missionary Service', 'original_title': 'Preach My Gospel (A Guide to Missionary Service)'}]], 'rating': [4.07646881936534]}
Wall time: 26.9 s


# Visualise

# todo : check

In [63]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, n_iter=500, verbose=3, random_state=1)
books_embedding = tsne.fit_transform(svd.qi)
projection = pd.DataFrame(columns=['x', 'y'], data=books_embedding)
projection['title'] = df_b['original_title']

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 10000 samples in 0.002s...
[t-SNE] Computed neighbors for 10000 samples in 2.215s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 0.243588
[t-SNE] Computed conditional probabilities in 0.325s
[t-SNE] Iteration 50: error = 102.5939636, gradient norm = 0.1709282 (50 iterations in 2.297s)
[t-SNE] I

In [66]:
import plotly.express as px

fig = px.scatter(
    projection, x='x', y='y'
)
fig.show()

In [71]:
import datapane as dp

def plot_books(titles, plot_name):
    
    book_indices = []
    for book in titles:
        book_indices.append(get_book_id(book, df_b)-1)
        
    book_vector_df = projection.iloc[book_indices]
    
    fig = px.scatter(
    book_vector_df, x='x', y='y', text='title',
    )
    fig.show()
    
    report = dp.Report(dp.Plot(fig)) #Create a report
    report.publish(name=plot_name, open=True, visibility='PUBLIC')

In [81]:
books = list(df_b['title'][:500])
plot_books(books, plot_name='books_embedding')

[15:43:46] [DEBUG] Saved object to D:\code\cours\books_recommandation\dp-cache\dp-tmp-i0rurilt\dp-tmp-ygvy9tu5.pl.json (40654 bytes)


Publishing report and associated data - please wait..

[15:43:46] [DEBUG] Successfully Built Report
[15:43:46] [DEBUG] Refreshed https://datapane.com/api/reports/BAmNb13/
[15:43:46] [DEBUG] Proxying 'web_url' lookup to DTO
[15:43:46] [DEBUG] Proxying 'web_url' lookup to DTO


Report successfully published at https://datapane.com/u/tjohanne42/reports/books-embedding/

In [75]:
!datapane login --token=8a45416f8267d1abda7b8d2c8764da3e337b913a

Connected successfully to https://datapane.com as tjohanne42


In [76]:
dp.enable_logging()

Configuring datapane logging in library mode
